<h1>Lexington Comment Prep</h1>

<br>
Read source file from survey and output the set of files which will be used for data processing.
<br>
Because comments are intermixed in source file, all output files are generated from this one process.


In [ ]:
%autosave 30
import sys
import os
import pandas as pd

in_file = os.path.join(os.path.join('..','src_data'),
                       'town services and housing costs.csv')
df = pd.read_csv(in_file, encoding='utf-8')

# Note: three fields had trailing spaces
rename_dict = \
{'In the past 12 months, has any household member done the following? (Check all that apply) [Visited the Community Center]':'ActivityComCenter',
'In the past 12 months, has any household member done the following? (Check all that apply) [Used town swimming facilities]':'ActivitySwim',
'In the past 12 months, has any household member done the following? (Check all that apply) [Used town recreational fields or courts]':'ActivityField',
'In the past 12 months, has any household member done the following? (Check all that apply) [Used a school or neighborhood playground]':'ActivityPlayground',
'In the past 12 months, has any household member done the following? (Check all that apply) [Used the Minuteman Bikeway]':'ActivityMinutemanBikeway',
'In the past 12 months, has any household member done the following? (Check all that apply) [Visited Lexington conservation areas]':'ActivityConservation',
'In the past 12 months, has any household member done the following? (Check all that apply) [Directly received fire or police services]':'ActivityFirePolice',
'In the past 12 months, has any household member done the following? (Check all that apply) [Attended Lexington public schools]':'ActivityLPS',
'In the past 12 months, has any household member done the following? (Check all that apply) [Participated in any town provided program or service for seniors]':'ActivitySenior',
'In the past 12 months, has any household member done the following? (Check all that apply) [Used Cary library or attended a library event]':'ActivityCary',
'In the past 12 months, has any household member done the following? (Check all that apply) [Served town government in any capacity (employee, committee member, town meeting member, elected representative)]':'ActivityTown',
'Are you aware that the Recreation & Community Programs Department offers financial aid for programs and services?':'RecreationAidAware',
'Has a household member received this type of financial assistance to participate in a program or service? ':'RecreationAidReceive',
'If your primary residence is rental, please indicate your monthly rental cost: ':'MonthlyRent',
'If you own your primary residence, please indicate your total monthly housing payment (mortgage, insurance, property taxes):':'MonthlyHousingCost',
'Do you receive a property tax deferral or property tax credit due to limited income?':'TaxDeferral',
'Does the homeowner own additional properties (vacation home, rental home, condominium, etc.)?':'Unused1',
'What level of stress does your household experience with payment of monthly housing costs?':'Stress',
'Thinking about all the costs of living in your owned property or rental (rent/mortgage, home maintenance, condo fees, property taxes, insurance, utilities), what portion of these costs do you estimate is related to property taxes?':'PropertyTaxShare',
'Are you considering relocating away from Lexington in the next 10 years?':'Relocation',
'Age (respondent)':'Age',
'Size of household (number of people)':'HouseholdSize',
'Disabilities within household (check all that apply)':'Disabilities',
'Length of time living in town (respondent)':'Tenure',
'What type of property is your residence?':'PropertyType',
'Approximate market property value of your Lexington residence (if owned):':'MarketValue',
'Please indicate your approximate household income in the past 12 months':'Income',
'Race/ethnicity (check all that apply)':'Unused2',
'Please indicate how the survey was shared with you (any groups or lists or whether it was forwarded to you.)':'ORShared',
'Please share any comments to the survey team about this survey. Thank you.':'ORComments',
'In the past 12 months, has any household member done the following? (Check all that apply) [Used Lexpress bus]':'ActivityLexpress',
'Are you responsible for the housing costs at your residence (i.e. owner or lessee)?':'Responsible',
'In the past 12 months, has any household member done the following? (Check all that apply) [Used Pine Meadows golf course]':'ActivityGolf',
'Please elaborate or share any additional thoughts. ':'ORAdditional',
}

df.rename(columns=rename_dict, inplace=True)
print(df.info())

<h3>Corrections</h3>

In [ ]:

#
# clean up errors
#
# 1. MonthlyHousingCost we don't trust...
# I think this needs to be done before the classification

# 
# we should modify this soon to be two conditions, but lets see how this works
df.loc[(df['MonthlyHousingCost']=='$10,001+') & 
       (df['MarketValue']=='$500,000-$749,999'),'MonthlyHousingCost']='ResponseError'
df.loc[(df['MonthlyHousingCost']=='$10,001+') & 
       (df['MarketValue']=='$750,000-$999,999'),'MonthlyHousingCost']='ResponseError'
df.loc[(df['MonthlyHousingCost']=='$10,001+') & 
       (df['MarketValue']=='Less than $500,000'),'MonthlyHousingCost']='ResponseError'
df.loc[(df['MonthlyHousingCost']=='$10,001+') & 
       (df['MarketValue']=='$1,000,000-$1,249,999'),'MonthlyHousingCost']='ResponseError'
df.loc[(df['MonthlyHousingCost']=='$10,001+') & 
       (df['MarketValue']=='$1,250,000-$1,499,999'),'MonthlyHousingCost']='ResponseError'
df['MonthlyHousingCost'].value_counts()


<H3>Data Driven Transformations</h3>

In [ ]:
transforms = [
    [
    'Tenure', 'TenureStat', None,
{'moved in 1979 and earlier':1975,
'moved in 1980-1989':1985,
'moved in 1990-1999':1995,
'moved in 2000-2009':2005,
'moved in 2010-2014':2012,
'moved in 2015 or later':2017}
    ],
    [
    'Tenure', 'TenureCode', None,
{'moved in 1979 and earlier':1979,
'moved in 1980-1989':1980,
'moved in 1990-1999':1990,
'moved in 2000-2009':2000,
'moved in 2010-2014':2010,
'moved in 2015 or later':2015}
    ],
    ['Stress', 'StressCode', None,
    {'None': 0,
    'Little': 1,
     'Some':2,
    'Significant':3,
    'Substantial':4}
],
    ['Stress', 'StressCode2', None,
    {'None': 0,
    'Little': 0,
     'Some':1,
    'Significant':2,
    'Substantial':2}
],
     ['StressCode2', 'StressBand', None,
    { 0 : '0. Low Stress',
     1 : '1. Medium Stress',
     2 : '2. High Stress'}
],
    ['Stress', 'StressCodeDescriptive', None,
    {'None': '0. None',
    'Little': '1. Little',
     'Some': '2. Some',
    'Significant': '3. Significant',
    'Substantial': '4. Substantial'}
],

    ['MarketValue','MarketValueStat',None,
    {
        '$1,000,000-$1,249,999':1125000,
'$1,250,000-$1,499,999':1375000,
'$1,500,000-$1,749,999':1625000,
'$1,750,000-$1,999,999':1875000,
'$2,000,000 and more':  2500000,
'$500,000-$749,999':625000,
'$750,000-$999,999':875000,
'Less than $500,000':400000,
    }
    ],
    ['MarketValue','MarketValueCode',None,
    {
        '$1,000,000-$1,249,999':1000000,
'$1,250,000-$1,499,999':1250000,
'$1,500,000-$1,749,999':1500000,
'$1,750,000-$1,999,999':1750000,
'$2,000,000 and more':  2000000,
'$500,000-$749,999':500000,
'$750,000-$999,999':750000,
'Less than $500,000':0,
    }
    ],
    ['MarketValue','MarketValueCode1', None,
    {
        '$1,000,000-$1,249,999':1000000,
'$1,250,000-$1,499,999':1250000,
'$1,500,000-$1,749,999':1500000,
'$1,750,000-$1,999,999':1750000,
'$2,000,000 and more':1750000,
'$500,000-$749,999':0,
'$750,000-$999,999':750000,
'Less than $500,000':0,
    }
    ],
    ['Income', 'IncomeStat', None,
     {
         '$100,000-$149,999':125000,
'$150,000-$199,999':175000,
'$200,000 and over':300000,
'$35,000-$49,999':37500,
'$50,000-$74,999':62500,
'$75,000-$99,000':87500,
'Less than $35,000':25000,
     }
    ],
    ['Income', 'IncomeCode', None,
     {
         '$100,000-$149,999':100000,
'$150,000-$199,999':150000,
'$200,000 and over':200000,
'$35,000-$49,999':35000,
'$50,000-$74,999':50000,
'$75,000-$99,000':75000,
'Less than $35,000':0,
     }
    ],
    ['Income', 'IncomeCode1', None,
    {
        '$100,000-$149,999':100000,
'$150,000-$199,999':150000,
'$200,000 and over':200000,
'$35,000-$49,999':0,
'$50,000-$74,999':50000,
'$75,000-$99,000':75000,
'Less than $35,000':0,

    }
    ],
    ['Income', 'IncomeCode2', None,
    {
        '$100,000-$149,999':100000,
'$150,000-$199,999':100000,
'$200,000 and over':200000,
'$35,000-$49,999':0,
'$50,000-$74,999':0,
'$75,000-$99,000':0,
'Less than $35,000':0,

    }
    ],
    ['Age','AgeStat',None,
{    
    '31-39 years':35,
'40-49 years':45,
'50-59 years':55,
'60-69 years':65,
'70-79 years':75,
'80 years and over':85,
'under 30 years':25,
}],
    ['Age','AgeCode',None,
{    
    '31-39 years':30,
'40-49 years':40,
'50-59 years':50,
'60-69 years':60,
'70-79 years':70,
'80 years and over':80,
'under 30 years':0,
}],
    ['Age','AgeCode1',None,
{    
'31-39 years':0,
'40-49 years':40,
'50-59 years':50,
'60-69 years':60,
'70-79 years':70,
'80 years and over':70,
'under 30 years':0,
}
    ],
    ['Age','AgeCode2',None,
{    
'31-39 years':0,
'40-49 years':0,
'50-59 years':50,
'60-69 years':60,
'70-79 years':70,
'80 years and over':70,
'under 30 years':0,
}],
    ['MonthlyRent','MonthlyRentStat',None,
       { '$0-$1499':750,
'$1500-2999':2250,
'$3000-$4499':3750,
'$4500+':5500}
    ],
    ['MonthlyRent','MonthlyRentCode',None,
       { '$0-$1499':0,
'$1500-2999':1500,
'$3000-$4499':3000,
'$4500+':4500}
    ],
    ['MonthlyHousingCost','MonthlyHousingStat',None,
        {
            '$10,001+':11000,
'$2,001-$4,000':3000,
'$4,001-$6,000':5000,
'$6,001-$8,000':7000,
'$8,001-$10,000':9000,
'<$2,000/month':1000,
        }],
    ['MonthlyHousingCost','MonthlyHousingCostCode',None,
        {
            '$10,001+':10001,
'$2,001-$4,000':2001,
'$4,001-$6,000':4001,
'$6,001-$8,000':6001,
'$8,001-$10,000':8001,
'<$2,000/month':0,

        }
    ],
    ['MonthlyHousingCost', 'MonthlyHousingCostCode1', None,
    {
        '$10,001+':6001,
'$2,001-$4,000':2001,
'$4,001-$6,000':4001,
'$6,001-$8,000':6001,
'$8,001-$10,000':6001,
'<$2,000/month':0,
    }
    ],
    ['Responsible', 'ResponsibleCode', None,
     { 'No': 0,
     'Yes': 1}
    
    ],
    ['Relocation', 'RelocationCode', None,
{
'Definitely Not':0, 
'Unlikely': 1 ,    
'Have not considered': 2 ,
'Undecided - Possible': 3,
'Likely': 4   ,           
'Definitely': 5 ,          
} 
    ],
    ['Relocation', 'RelocationStat1', None,
{
'Definitely Not':0 ,
'Unlikely': 1   ,  
'Have not considered': 2 ,
'Undecided - Possible': 2,
'Likely': 3 ,            
'Definitely': 4,          
} 
    ],
    ['Relocation', 'RelocationStat2', None,
{
'Definitely Not':0 ,
'Unlikely': 0   ,  
'Have not considered': 1 ,
'Undecided - Possible': 1,
'Likely': 2 ,            
'Definitely': 2,          
} 
    ],
    ['Relocation', 'RelocationStat3', None,
{
'Definitely Not':0 ,
'Unlikely': 1   ,  
'Undecided - Possible': 2,
'Likely': 3 ,            
'Definitely': 4,          
} 
    ],
    ['Relocation', 'RelocationStat4', None,
{
'Definitely Not':0 ,
'Unlikely': 0   ,  
'Undecided - Possible': 1,
'Have not considered': 1 ,
'Likely': 2 ,            
'Definitely': 2,          
} 
    ],
    ['RelocationStat2', 'RelocationBand', None,
{
0: 'Unlikely' ,
1: 'Neutral' ,  
2: 'Likely',          
} 
    ],
    ['PropertyTaxShare', 'PropertyTaxShareStat', None,
     {
         'Some (26-50%)' : 38,
    'Significant (51-75%)' : 63,
    'Little (1-25%)' : 13,
    'Substantial (76-100%)' : 87,
    'None' : 0 
     }
    ],
    ['PropertyTaxShare', 'PropertyTaxShareCode', None,
     {
         'Some (26-50%)' : 26,
    'Significant (51-75%)' : 51,
    'Little (1-25%)' : 1,
    'Substantial (76-100%)' : 76,
    'None' : 0 
     }
    ],
    ['HouseholdSize', 'HouseholdSizeStat', None,
        { '1': 1,
        '2':2,
        '3':3,
        '4':4,
        '5+' : 5}
    ]
]


print('Generating transformations')
for transform in transforms:
    from_col, to_col, default_value, transform_dict = transform
    print('%s: %s' % (from_col, to_col))
    df[to_col]=default_value
    for key in transform_dict.keys():
        df.loc[df[from_col] == key, to_col] = transform_dict[key]
    print(df[to_col].value_counts().sort_index())  
    print(f'Total responses: {df[to_col].count()}')

 
stat_means = [
    ['IncomeStat' , 'IncomeStatMean'],
    ['AgeStat', 'AgeStatMean'],
    ['MarketValueStat', 'MarketValueStatMean'],
    ['IncomeStat', 'IncomeStatMean'],
    ['TenureStat', 'TenureStatMean'],
    ['HouseholdSizeStat', 'HouseholdSizeStatMean'],
    ['StressCode','StressCodeStatMean'],
    ['MonthlyHousingStat', 'MonthlyHousingStatMean'],
    ['RelocationStat1','RelocationCodeStat1Mean'],
    ['RelocationStat2','RelocationCodeStat2Mean'],
    ['RelocationStat3','RelocationCodeStat3Mean'],
    ['RelocationStat4','RelocationCodeStat4Mean'],
    ['PropertyTaxShareStat','PropertyTaxShareStatMean'],
    ['MonthlyRentStat', 'MonthlyRentStatMean']
]

print('Generating statistical means')
for stat_mean_gen in stat_means:
    from_col, to_col = stat_mean_gen
    df[to_col] = df[from_col] - df[from_col].mean()
    print(f'{to_col}: {df[to_col].mean()}')

own_resp_stat_means = [
    ['IncomeStat' , 'IncomeORStatMean'],
    ['AgeStat', 'AgeORStatMean'],
    ['MarketValueStat', 'MarketValueORStatMean'],
    ['IncomeStat', 'IncomeORStatMean'],
    ['TenureStat', 'TenureORStatMean'],
    ['HouseholdSizeStat', 'HouseholdSizeORStatMean'],
    ['StressCode','StressCodeORStatMean'],
    ['MonthlyHousingStat', 'MonthlyHousingORStatMean'],
    ['RelocationStat1','RelocationCodeORStat1Mean'],
    ['RelocationStat2','RelocationCodeORStat2Mean'],
    ['RelocationStat3','RelocationCodeORStat3Mean'],
    ['RelocationStat4','RelocationCodeORStat4Mean'],
    ['PropertyTaxShareStat','PropertyTaxShareORStatMean'],
    ['MonthlyRentStat', 'MonthlyRentORStatMean']
]

print('OwnRent must be generated prior to statistical means adj for responsible and OwnRent')
# Own or Rent
df['OwnRent'] = '-1'
df.loc[df['MarketValueCode']>=0, 'OwnRent'] = 'Own'
df.loc[df['MonthlyRentCode']>=0, 'OwnRent'] = 'Rent'

print('Own or Rent')
print(df['OwnRent'].value_counts())

print('Generating owner responsible statistical means')
responsibleowner_set = df[((df['ResponsibleCode']==1) & (df['OwnRent']=='Own'))]
for stat_mean_gen in own_resp_stat_means:
    from_col, to_col = stat_mean_gen
    adj_mean = responsibleowner_set[from_col].mean()
    df[to_col] = df[from_col] - adj_mean
    print(f'{to_col}: {df[to_col].mean()} count {df[to_col].count()}')

    

<h3>Code Driven Transformations</h3>

In [ ]:
print(df['ResponsibleCode'].value_counts())


# Means Tested Approximation - responsible for housing
df['MeansTestedApprox'] = -1
#
# owners who filled in questions only
df.loc[(df['OwnRent']=='Own') &
      (df['ResponsibleCode']==1) &
       (df['TenureCode']!=-1)&
       (df['IncomeCode1']!= -1) &  
       (df['AgeCode']!=-1) &
       (df['MarketValueCode']!=-1), 'MeansTestedApprox'] = 0
# subset who meet the criteria
df.loc[(df['MeansTestedApprox']==0) & (df['MarketValueCode']<=750000) &
       (df['AgeCode']>=60) & (df['IncomeCode1']<=60000) & (df['TenureCode']<=2005) , 'MeansTestedApprox'] = 1
print('MeansTestedApproximation')
print(df['MeansTestedApprox'].value_counts())

# testing means tested
print('Means tested testing')
mt_df = df[df['MeansTestedApprox']==1]
for field in ['MarketValue', 'MarketValueCode', 'Age', 'AgeCode', 'Income', 'IncomeCode1', 'Tenure', 'TenureCode']:
    print(field)
    print(mt_df[field].value_counts())
    


df['ActivityCount'] = 0
activity_list = ['ActivityMinutemanBikeway', 'ActivityCary', 'ActivityComCenter', 'ActivityConservation', 
                 'ActivityField', 'ActivityFirePolice', 'ActivityGolf', 'ActivityLPS', 'ActivityLexpress', 
                 'ActivityPlayground', 'ActivitySenior', 'ActivitySwim', 'ActivityTown']
for activity in activity_list:
    df.loc[df[activity]=='Yes', 'ActivityCount'] += 1

print('ActivityCount')   
print(df['ActivityCount'].value_counts().sort_index())

    
# Document Transformations
doc_file = 'E:\\LexingtonSurvey\\doc.txt'
with open(doc_file, 'w') as fhandle:
    fhandle.write('Transformations Documentation\n')
    for transform in transforms:
        from_col, to_col, default_value, transform_dict = transform
        out_str = '%s: from %s\n' % (to_col, from_col)
        fhandle.write(out_str)
        fhandle.write(' Mappings:\n')
        for ky in sorted(transform_dict.keys()):
            out_str = '  %s: %s\n' % (ky, str(transform_dict[ky]))
            fhandle.write(out_str)
        fhandle.write(' Frequency Distribution:\n')
        out_str = df[to_col].value_counts()
        fhandle.write('%s\n\n' % out_str)
       
    for stat_mean_gen in stat_means:
        from_col, to_col = stat_mean_gen
        fhandle.write('Generating statistical mean variable %s by subtracting mean from %s.\n' % (to_col, from_col))
    fhandle.write('\n\nManual Transformations:')
    fhandle.write('OwnRent: Whether individual is coded as owner or renter\n')
    fhandle.write(' Rental Cost must be filled in to be counted as renter.\n')
    fhandle.write(' Market value must be filled in to be counted as owner.\n')
    fhandle.write(' Frequency Distribution:\n')
    out_str = df['OwnRent'].value_counts()
    fhandle.write('%s\n\n' % out_str)
    fhandle.write('MeansTestedApprox: Whether individual approximates criteria for means tested exemption\n')
    fhandle.write('Must be responsible for housing and own, and have filled in the following survey questions:\n')
    fhandle.write(' Tenure in Lexington, Income, Age, Market Value of Home\n')
    fhandle.write('If all quesions are filled in, respondent will be coded 0 or 1.\n')
    fhandle.write('To be coded 1 also requires: age>=60, Income<=$75,000, Arrival in Lexington < 2010\n')
    fhandle.write('Market Value of Home <= $999,999.\n')
    fhandle.write(' Frequency Distribution:\n')
    out_str = df['MeansTestedApprox'].value_counts()
    fhandle.write('%s\n\n' % out_str)
    fhandle.write('ActivityCount: Number of activities participated by household member in prior 12 months\n')
    fhandle.write('Summed the answers to each activity question, range 0-13.\n')
    fhandle.write(' Frequency Distribution:\n')
    out_str = df['ActivityCount'].value_counts()
    
print('done')   
        

In [ ]:
print(sorted(list(df.columns)))

<h3>Clean output files for Public Use</h3>

In [ ]:
from sklearn.utils import shuffle
public_cols = list(df.columns)

print('Preparing for export of public data set')
for col in ['ORShared','ORComments','ORAdditional','Timestamp']:
    print('Removing column %s' % col)
    public_cols.remove(col)

public_cols.sort()
print('Exported columns inlude:')
print(public_cols)
# change order of rows to avoid people reading into the ordering of the rows
print('Shuffling data row order')
df_clean = shuffle(df[public_cols])
out_file =  os.path.join('lexsurveyfiles','town services and housing costs public.xlsx')
print('Outputting file %s' % out_file)
df_clean.to_excel(out_file, index=False, encoding='utf-8')
print('done')

In [ ]:
#
# Generating alternative set with one row per person-activity
#
# start with public columns: which is df_clean's list

non_activity_columns = list(set(df_clean.columns) - set(activity_list))
print(non_activity_columns)

df_list = []
for activity in activity_list:
    activity_name = activity[8:]
    print(activity_name)
    # put in the word 'Blank' if not filled out
    df_clean.loc[df_clean[activity].isnull()==True, activity] = 'Blank'
    print(df_clean[activity].count())
    
    df_one_act = df_clean[non_activity_columns + [activity]].copy()
    df_one_act.loc[:, 'ActivityKey'] = activity_name
    df_one_act.rename(columns={activity:'ActivityValue'}, inplace=True)
    df_list.append(df_one_act)
    print(df_one_act['ActivityValue'].value_counts())
    
df_activities = pd.concat(df_list)
#print df_activities.ActivityKey.value_counts()
print(df_activities.ActivityValue.value_counts())

df_activities = df_activities[sorted(list(df_activities.columns))]
out_file = os.path.join('lexurveyfiles','town services and housing costs public activities.xlsx')
print('Outputting file %s' % out_file)
df_activities.to_excel(out_file, index=False, encoding='utf-8')
print(df_activities.info())



In [ ]:
#
# manual redactions
#
or_additional_replacements = [
['retired school teachers living on TIAA-CREF savings and Social Security', 
     'retired [living on retirement savings] and Social Security'],
    ['I bought a small 2 BR cape from a friend 20 years ago- I had an unusual break. I had been recently laid off  and I was the wife of a public school teacher',
 'I bought a small [house] from a friend 20 years ago- I had an unusual break. I had been recently laid off [...],'],
['age 64', 'age 6[x]'],
["I sold my house to a Chinese family with two kids- one in pre-school.", 
     'I sold my house [...].'],
["speak English well.", '[...]'],
['m 60 and i', 'm 6[x] and i'],
['at age 70', 
     'at age 7[x]'],
['Brookline', 
     '[peer town]'],
['former employee of LPS', 
     'former [public employee]'], 
['corporate lawyer and the higher income', \
     '[]'],    
['in Somerville', 
     '[]'],
['only 63', 'only 6[x]'],
['age 63', 'age 6[x]'],
['Despite me having 3 masters degrees and a 20 year solid career as a clinical mental health counselor .', \
     '[].'],  
['semi-retired (65)', 'semi-retired (6[x])'],
['two daughters (8th and 10th grade)', 'two [children in school]'],
['until our daughters graduate', 'until our [children] graduate'],
['nine', '[]']
]

or_comments_replacements = [
    ['was born in 1952 in lexington', 'was born in 195[x] in Lexington'],
    ['Venezuelan immigrant', '[foreign country] immigrant'],
    ['daughter just started kindergarten', '[child just started school]'],
    ['daughter just started kindergarten', '[child just started school]'],
    ['My 28-year-old-son', 'My 2[x-year-old-child]'],
    ['sold my loan to Chase', 'sold my loan to [company]'],
    ['two post-high-school sons at home', '[multiple post-high-school children] at home'],
    ['for pay since 2000', 'for pay since 20[xx]'],
    ['1965', '196[x]'],
    ['46', '4[x]']
]

or_shared_replacements = [
    ['markandersen@alum.mit.edu/Yahoo group', '[Lexington yahoo group]'],
    ["Sanjay Padaki's email", '[Personal email]'],
    ['liberty-lex email group shared by Joe Pato.', 'liberty-lex email group [...]'],
    ['Lexington at Home publicity Chair- Howard Cloth', 'Lexington at home'],
    ['Lexington List via Mark Andersen', 'Lexington List [...]'],
    ['recdept@lexingtonma.gov', '[Recreation Department]'],
    ['Joe Pato via Google groups Liberty Heights', '[...] Liberty Heights'],
    ['Joe Pato through LexLiberty List serve.', '[...] LexLiberty List serve'],
    ['Vicki Blier shared a Facebook post on Battle Road Action Committee Group', '[...] Battle Road Action Committee Group'],
    ['Email from selectman Joe Pato', 'Email from selectman [...]'],
    ['lexingtonfriends@googlegroups.com', '[lexington friends google group]'],
    ['Lexington Human Services <hshah@lexingtonma.gov>', 'Lexington Human Services [...]'],
    ['Lexington Human Services hshah@lexingtonma.gov', 'Lexington Human Services [...]'],
    ["Harry's Lexington email list", '[...] Lexington email list'],
    ['lexingtonfriends@yahoo.com', '[Lexington friends yahoo group]'],
    ['LexingtonFriends email forwarded by John Zhao', 'LexingtonFriends email forwarded by [...]'],
    ['Town Meeting rep Sanjay Padaki', 'Town Meeting rep [...]'],
    ['Emsil', '[Email]'],
    ['Selectman Joe Pato', 'Selectman [...]'],
    ["Joe Pato's facebook page", '[Selectman xxx] facebook page'],
    ['The Lexington List (lexington@yahoogroups.com)', 'The Lexington List [...]'],
    ['Joe Pato (Selectman) sent it.', '[Selectman]'],
    ['hshah@lexingtonma.gov via auth.ccsend.com', '[Lexington human services]'],
    ['Sanjay Padaki TMM, Precinct 8', '[Town Meeting Member]'],
    ['Lexington List email (Harry Forsdick moderates)', 'Lexington List email [...]'],
    ['from Joe Pato', 'from [Selectman]'],
    ['Lex Rec recdept@lexingtonma.gov', 'Lexington [Recreation Department]'],
    ["Joe Pato's email", '[Selectman email]'],
    ['Next Door list posting by Joe Pato', 'Next Door list posting [...]'],
    ['Lexington@yahoogroups.com', '[Lexington yahoo group]'],
    ['Sanjay padki', '[Town Meeting Member]'],
    ['Joe Pato', '[Selectman]'],
    ['From John Zhao in Lexington Chinese Wechat Group', '[...] Lexington Chinese Wechat Group'],
    ['The survey was forwarded by John Zhao.', 'The survey was forwarded by [...]'],
]

# the plain Joe Pato should be last so it catches remaining


for replacements, field in [[or_additional_replacements, 'ORAdditional'], 
                            [or_comments_replacements, 'ORComments'],
                           [or_shared_replacements, 'ORShared']]:
    print('----- Field %s ------' % field)
    for replacement in replacements:
        old_str, new_str = replacement
        print('Replacing %s' % old_str)
        df[field] = df[field].str.replace(old_str, new_str, regex=False)
        # check
        print('%s:' % old_str)
        clean_count = (df[df[field].str.contains(old_str)==True][field].count())
        if clean_count > 0:
            raise('Failure to remove text in cleanup')
#
# create output files
#
for col in ['ORShared','ORComments','ORAdditional']:
    out_file = os.path.join('lexsurveyfiles', 'town services and housing costs public activities %s' % col)
    print('Outputting file %s' % out_file)
    
    # get column without blank rows
    df_out = df.loc[df[col].isnull() == False, [col]]
    df_out = shuffle(df_out)
    
    df_out.to_excel(out_file + '.xlsx', index=False, header=False, encoding='utf-8')
    df_out.to_csv(out_file + '.txt', index=False, header=False, encoding='utf-8')
